In [ ]:
#实时获取数据并输入模型
import json
import socket
from keras.models import load_model
import tensorflow as tf
#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
import pattlib as pm
#将骨骼坐标数据转化为训练数据
import cv2
import numpy as np
import time
import json
from scipy import signal

pattern=[118.8589540646638, 125.53607139836588, 133.26291685014698, 151.0608954265668, 162.9591136621205, 168.13987020349498, 173.3011197194798, 177.80499893572988, 177.80499893572988, 171.9251772639162, 172.85086814947013, 164.76053906589502, 156.69984911240775, 139.08440986657644, 129.0761349679326, 115.6391075863956]
for i in range(16):
    pattern[i]=pattern[i]-8
#data_D=len(keypoints)
def listsub(e=[],r=[]):
    n=[]
    for i in range(2):
        u=e[i]-r[i]
        n.append(u)
    return n

def speed_evaluate(list):
    #print(list)
    re=0
    for i in range(len(list)-1):
        if abs(list[i]-list[i+1])>12:
            re = 1
            break
    
    return re
    

def co_convert(co):
    for i in range(len(co)):
        if co[i]!=[0,0]:
            co[i]=listsub(co[i],co[17])
    return co

def save_to_json(data):
    with open('data1.json', 'a') as file:
        json.dump(data, file)
        file.write('\n')
        
def list_to_string(lst):
    str_lst = [str(element) for element in lst]  # 将列表中的每个元素转换为字符串
    result = ','.join(str_lst)  # 使用空字符串连接所有字符串元素
    result = '['+result+']'
    return result

def getframework(co):
    a=co[5]
    a[1]=a[1]-15
    coo=a
    h=co[15][1]-co[5][1]
    return coo,h
    

def convert(keypoints):
    # = int(object_counts[0])
    #K = topology.shape[0]
    
    #color = (0, 255, 0)
    sk=[]
    k=co_convert(keypoints[0])
    #print(k)
    for j in range(18):
        if 4<j<17:
            d = k[j]
        #if k != [0,0]:
            sk.append(d[0])
            sk.append(d[1]) 
    return sk

def show_re(image,data_,lis):
    if len(data_)==50:
            z=np.mat(data_)
            z=np.expand_dims(z, axis=0)
            re=model(z)
            re=re.numpy()
            re=re[0]
            oi=np.where(re==np.max(re))
            io=np.max(re)
            oi=list(oi)
            oi=oi[0][0]
            oii=oi
                        #print(type(io))
            
            oi=str(oi)
                        #image = np.zeros([224, 224, 3], np.uint8)+255 
            if io>0.5:
                cv2.putText(image,oi,(0,20),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)
                io=str(io)
                cv2.putText(image,io,(0,40),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)
                if oii==0:
                    cv2.putText(image,'side lateral raise is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                    re1,re2 = pm.pattern_matching(data_,pattern)
                    re1=np.array(re1)
                    peak_indexes = signal.argrelextrema(re1, np.less, order=10)
                    peak_indexes = peak_indexes[0]
                    
                    if peak_indexes.size>0:
                        peak_indexes=min(peak_indexes)
                        edge = 50-peak_indexes-1
                        
                        ede = data_[edge-16:edge]
                        #print(ede)
                        x,y_l,y_r = pm.get_angle_list(ede)
                        #print(max(y_l))
                    #x,y,z = get_angle_list(ede) 
                    #peak_indexes=max(y)
                    #peak_indexes = str(peak_indexes)
                        max_angle_l=max(y_l)
                        max_angle_r=max(y_r)
                        min_angle_l=min(y_l)
                        min_angle_r=min(y_r)
                        #print(min_angle_l)
                        spe=speed_evaluate(y_l)
                        #print(spe)
                        if spe==1:
                            cv2.putText(image,'speed so fast',(0,180),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
                        else:
                            cv2.putText(image,'good speed',(0,180),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                        if max_angle_l<160:
                            cv2.putText(image,'insufficient arm raising',(0,210),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
                        else:
                            cv2.putText(image,'sufficient arm raising',(0,210),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                        cv2.putText(image,'max angle:',(0,140),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                        cv2.putText(image,str(max_angle_l),(110,140),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                    
                if oii==1:
                    cv2.putText(image,'bicep curl is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                if oii==2:
                    cv2.putText(image,'dumbell front raise is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                if oii==3:
                    cv2.putText(image,'dumbbell squat is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                            #print(lst[0])
                image = cv2.rectangle(image, lis[0][5], lis[0][12], (255, 0, 0), 1)
                            #print(lis[0][5])
    cv2.namedWindow("frame",cv2.WINDOW_NORMAL)
    cv2.imshow("frame",image)

def start_server(host, port, model):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    
    #cap = cv2.VideoCapture(0)

# 设置摄像头帧率为10帧每秒
    #cap.set(cv2.CAP_PROP_FPS, 15)
    
# 设置摄像头分辨率为224x224
    #cap.set(3, 224)  # 设置宽度
    #cap.set(4, 224)  # 设置高度

    print(f"服务器正在监听 {host}:{port}")
    data_ = []
    buffer = b''
    while True:
        
        client_socket, client_address = server_socket.accept()
        print(f"与客户端 {client_address[0]}:{client_address[1]} 建立连接")

        while True:

            try:
                
                
                received_data = client_socket.recv(1024)  # 1024是缓冲区大小，你可以根据实际需求调整
                buffer += received_data
                while b'#' in buffer:
                    # 分割消息
                    index = buffer.index(b'#')
                    message = buffer[:index]
                    buffer = buffer[index + 1:]
                    print("1")
                    # 处理接收到的消息
                    print(f"接收到的消息: {message.decode('utf-8')}")
                
                
                    data = message.decode().strip()
                #time.sleep(0.05)
                    if not data:
                        break

                    try:
                        da='['+data+']'
                        #da.append(data)
                        print(data)
                        #ret, image = cap.read()
                        image = np.ones((224, 224, 3), dtype=np.uint8) * 255
                        lis=json.loads(da)
                        lst = json.loads(da)
                        #print(lst)
                        z=convert(lst)
                        data_.append(z)
                    #print(len(data_))
                    
                    #image = cv2.resize(frame, (224, 224))
                        show_re(image,data_,lis)
                        if len(data_)==50:
                        
                            del(data_[0])
                    except json.JSONDecodeError:
                        print("接收到的数据不是有效的JSON格式")
                
            except ConnectionResetError:
                break
            if cv2.waitKey(1)&0xFF==ord("q"):
                break
                
        if not data or data.decode('utf-8') == 'exit':
            break
        
        print(f"与客户端 {client_address[0]}:{client_address[1]} 连接断开")
        client_socket.close()
    
        server_socket.close()
        break
# 用于测试的主函数
if __name__ == "__main__":
    host = '192.168.100.82'  # 服务器的主机地址
    port = 8817 # 服务器的端口号
    #del model 
    model = load_model('1Dcnnmodel.h5')
    start_server(host, port, model)

In [14]:
#实时获取数据并输入模型 加入心率获取testing
import json
import socket
from keras.models import load_model
import tensorflow as tf
#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
import pattlib as pm
#将骨骼坐标数据转化为训练数据
import cv2
import numpy as np
import time
import json
from scipy import signal
import threading

# 服务器地址和端口
vital_host = '192.168.100.82'
vital_port = 9906

pattern=[118.8589540646638, 125.53607139836588, 133.26291685014698, 151.0608954265668, 162.9591136621205, 168.13987020349498, 173.3011197194798, 177.80499893572988, 177.80499893572988, 171.9251772639162, 172.85086814947013, 164.76053906589502, 156.69984911240775, 139.08440986657644, 129.0761349679326, 115.6391075863956]
for i in range(16):
    pattern[i]=pattern[i]-8
#data_D=len(keypoints)
def listsub(e=[],r=[]):
    n=[]
    for i in range(2):
        u=e[i]-r[i]
        n.append(u)
    return n

def speed_evaluate(list):
    #print(list)
    re=0
    for i in range(len(list)-1):
        if abs(list[i]-list[i+1])>12:
            re = 1
            break
    
    return re
    

def co_convert(co):
    for i in range(len(co)):
        if co[i]!=[0,0]:
            co[i]=listsub(co[i],co[17])
    return co

def save_to_json(data):
    with open('data1.json', 'a') as file:
        json.dump(data, file)
        file.write('\n')
        
def list_to_string(lst):
    str_lst = [str(element) for element in lst]  # 将列表中的每个元素转换为字符串
    result = ','.join(str_lst)  # 使用空字符串连接所有字符串元素
    result = '['+result+']'
    return result

def getframework(co):
    a=co[5]
    a[1]=a[1]-15
    coo=a
    h=co[15][1]-co[5][1]
    return coo,h
    

def convert(keypoints):
    # = int(object_counts[0])
    #K = topology.shape[0]
    
    #color = (0, 255, 0)
    sk=[]
    k=co_convert(keypoints[0])
    #print(k)
    for j in range(18):
        if 4<j<17:
            d = k[j]
        #if k != [0,0]:
            sk.append(d[0])
            sk.append(d[1]) 
    return sk
vital = ''
def receive_vitaldata(host,port):
    # 创建UDP套接字
    udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    udp_socket.bind((host, port))
    global vital
    while True:
        # 接收数据
        data, client_address = udp_socket.recvfrom(1024)  # 1024是缓冲区大小，你可以根据实际需求调整
        message = data.decode('utf-8')
        vital = message
        # 处理接收到的数据
        print(f"接收到来自 {client_address} 的vital消息: {message}")


def show_re(image,data_,lis):
    global vital
    if len(data_)==50:
            cv2.putText(image,str(vital),(100,20),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)
            z=np.mat(data_)
            z=np.expand_dims(z, axis=0)
            re=model(z)
            re=re.numpy()
            re=re[0]
            oi=np.where(re==np.max(re))
            io=np.max(re)
            oi=list(oi)
            oi=oi[0][0]
            oii=oi
                        #print(type(io))
            
            oi=str(oi)
                        #image = np.zeros([224, 224, 3], np.uint8)+255 
            if io>0.5:
                cv2.putText(image,oi,(0,20),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)
                io=str(io)
                cv2.putText(image,io,(0,40),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)
                if oii==0:
                    cv2.putText(image,'side lateral raise is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                    re1,re2 = pm.pattern_matching(data_,pattern)
                    re1=np.array(re1)
                    peak_indexes = signal.argrelextrema(re1, np.less, order=10)
                    peak_indexes = peak_indexes[0]
                    
                    if peak_indexes.size>0:
                        peak_indexes=min(peak_indexes)
                        edge = 50-peak_indexes-1
                        
                        ede = data_[edge-16:edge]
                        #print(ede)
                        x,y_l,y_r = pm.get_angle_list(ede)
                        #print(max(y_l))
                    #x,y,z = get_angle_list(ede) 
                    #peak_indexes=max(y)
                    #peak_indexes = str(peak_indexes)
                        max_angle_l=max(y_l)
                        max_angle_r=max(y_r)
                        min_angle_l=min(y_l)
                        min_angle_r=min(y_r)
                        #print(min_angle_l)
                        spe=speed_evaluate(y_l)
                        #print(spe)
                        if spe==1:
                            cv2.putText(image,'speed so fast',(0,180),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
                        else:
                            cv2.putText(image,'good speed',(0,180),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                        if max_angle_l<160:
                            cv2.putText(image,'insufficient arm raising',(0,210),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
                        else:
                            cv2.putText(image,'sufficient arm raising',(0,210),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                        cv2.putText(image,'max angle:',(0,140),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                        cv2.putText(image,str(max_angle_l),(110,140),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                    
                if oii==1:
                    cv2.putText(image,'bicep curl is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                if oii==2:
                    cv2.putText(image,'dumbell front raise is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                if oii==3:
                    cv2.putText(image,'dumbbell squat is doing',(0,110),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1)
                            #print(lst[0])
                image = cv2.rectangle(image, lis[0][5], lis[0][12], (255, 0, 0), 1)
                            #print(lis[0][5])
    cv2.namedWindow("frame",cv2.WINDOW_NORMAL)
    cv2.imshow("frame",image)

def start_server(host, port, model):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    
    #cap = cv2.VideoCapture(0)

# 设置摄像头帧率为10帧每秒
    #cap.set(cv2.CAP_PROP_FPS, 15)
    
# 设置摄像头分辨率为224x224
    #cap.set(3, 224)  # 设置宽度
    #cap.set(4, 224)  # 设置高度

    print(f"服务器正在监听 {host}:{port}")
    data_ = []
    buffer = b''
    while True:
        
        client_socket, client_address = server_socket.accept()
        print(f"与客户端 {client_address[0]}:{client_address[1]} 建立连接")

        while True:

            try:
                
                
                received_data = client_socket.recv(1024)  # 1024是缓冲区大小，你可以根据实际需求调整
                buffer += received_data
                while b'#' in buffer:
                    # 分割消息
                    index = buffer.index(b'#')
                    message = buffer[:index]
                    buffer = buffer[index + 1:]
                    #print("1")
                    # 处理接收到的消息
                    #print(f"接收到的消息: {message.decode('utf-8')}")
                
                
                    data = message.decode().strip()
                #time.sleep(0.05)
                    if not data:
                        break

                    try:
                        da='['+data+']'
                        #da.append(data)
                        #print(data)
                        #ret, image = cap.read()
                        image = np.ones((224, 400, 3), dtype=np.uint8) * 255
                        lis=json.loads(da)
                        lst = json.loads(da)
                        #print(lst)
                        z=convert(lst)
                        data_.append(z)
                    #print(len(data_))
                    
                    #image = cv2.resize(frame, (224, 224))
                        show_re(image,data_,lis)
                        if len(data_)==50:
                        
                            del(data_[0])
                    except json.JSONDecodeError:
                        print("接收到的数据不是有效的JSON格式")
                
            except ConnectionResetError:
                break
            if cv2.waitKey(1)&0xFF==ord("q"):
                break
                
        if not data or data.decode('utf-8') == 'exit':
            break
        
        print(f"与客户端 {client_address[0]}:{client_address[1]} 连接断开")
        client_socket.close()
    
        server_socket.close()
        # 关闭套接字
        udp_socket.close()
        break
# 用于测试的主函数
if __name__ == "__main__":
    host = '192.168.100.82'  # 服务器的主机地址
    port = 8825 # 服务器的端口号
    #del model 
    model = load_model('1Dcnnmodel.h5')
    
    # 创建线程
    receive_vitalthread = threading.Thread(target=receive_vitaldata,args=(vital_host,vital_port))
    # 启动线程
    receive_vitalthread.start()
    start_server(host, port, model)
    


服务器正在监听 192.168.100.82:8825
与客户端 192.168.100.218:51728 建立连接
接收到来自 ('192.168.100.82', 51001) 的vital消息: HR:120 BR:20


KeyboardInterrupt: 